# Praktikum 1: masinõpe scikit-learni abiga

Moodle'is on üleval ka mõned CSV formaadis andmestikud, mida tänases praktikumis kasutame.   
Kui kasutad oma arvutis olevat Anaconda keskkonda, siis liiguta lihtsalt failid praktikumi Notebookiga samasse kausta.  
Kui kasutad Colaboratory, siis pane need oma Google Drive'i ja jooksuta läbi järgmine lahter, et see Notebookile külge võtta.

In [ ]:
#Failide lugemiseks Colabis
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/Colab Notebooks/ai")

Mounted at /content/drive


## Sissejuhatus

Tõenäosusliku modelleerimise eesmärgiks on leida mudel, mille põhjal võimalikult hästi ennustada seni veel nägemata sündmusi. Selleks valitakse andmete hulgast välja parameetrid, mis iseloomustavad ja üldistavad valimit võimalikult hästi, ning koostatakse (treenitakse) seni teadaolevate andmete põhjal mudel. 

Me püüame mõista, kuidas sõltub toas olemise või väljamineku otsus ilmastikust. Selleks loendame erinevaid juhte treeninghulgas ja säilitame andmed mitmemõõtmelises sõnastikus, kus võtmeteks on erinevad tingimuste ja otsuste väärtused, elementide väärtusteks aga loendamise tulemus, näiteks sõnastikuelemendi ["selge"]["toas"] väärtus 4 näitab, et kõigi meie andmete hulgas hulgas on 4 juhtu, kus väljas on olnud selge ilm ja inimene on otsustanud toas aega veeta. Samuti hoiame sõnastikus (võtmega "total") kõigi vastava tingimusega elementide hulka (kui palju esineb kokku selge ilmaga, palju pilves ilmaga juhtumeid), nende alusel saame pärast öelda, kui suur osakaal kõigi mingite tingimuste puhul on just sellisel otsusel. 

Tavaline Pythoni sõnastik sobiks loendamiseks samuti, aga kui me pole vastavate võtmetega elementi veel loonud, antakse meile viga. Seega peaksime loendamisel koodis pidevalt kontrollima: kui selliste võtmetega element on olemas, liida olemasolevale väärtusele 1, vastasel juhul loo selliste võtmetega uus element väärtusega 1. Et sellisest kontrollist pääseda, kasutame andmestruktuuri defaultdict.

In [ ]:
# Andmed: pilvisus (selge/pilves), temperatuur (soe/külm), otsus (toas/väljas)
trainData = [
    ["selge", "soe", "toas"],
    ["pilves", "soe", "väljas"],
    ["pilves", "külm", "toas"],
    ["selge", "külm", "toas"],
    ["selge", "soe", "väljas"],
    ["selge", "soe", "toas"],
    ["selge", "soe", "toas"],
    ["pilves", "külm", "toas"],
    ["pilves", "soe", "väljas"],
    ["pilves", "külm", "toas"],
]

from collections import defaultdict

# Loendame, mitu korda nii selge kui pilvise ilmaga otsustati toas olla või välja minna
pDecisionGivenCondition = defaultdict(lambda: defaultdict(float))

for sample in trainData:
    # Lisame õige elemendi väärtusele juurde 1
    # Näiteks esimese kirje puhul pDecisionGivenCondition["selge"]["toas"] += 1
    pDecisionGivenCondition[sample[0]][sample[2]] += 1
    # Samuti loendame ära kõik sama tingimusega ilma juhtumid
    pDecisionGivenCondition[sample[0]]["__total"] += 1
    
pDecisionGivenCondition

defaultdict(<function __main__.<lambda>()>,
            {'selge': defaultdict(float,
                         {'toas': 4.0, '__total': 5.0, 'väljas': 1.0}),
             'pilves': defaultdict(float,
                         {'väljas': 2.0, '__total': 5.0, 'toas': 3.0})})

Loome teise mudeli, mis arvestab temperatuuri ja tehtud otsuse seost.

In [ ]:
# Loendame, mitu korda nii selge kui pilvise ilmaga otsustati toas olla või välja minna
pDecisionGivenCondition1 = defaultdict(lambda: defaultdict(float))

for sample in trainData:
    # Lisame õige elemendi väärtusele juurde 1
    # Näiteks esimese kirje puhul pDecisionGivenCondition["soe"]["toas"] += 1
    pDecisionGivenCondition1[sample[1]][sample[2]] += 1
    # Samuti loendame ära kõik sama tingimusega ilma juhtumid
    pDecisionGivenCondition1[sample[1]]["__total"] += 1
    
pDecisionGivenCondition1

defaultdict(<function __main__.<lambda>()>,
            {'soe': defaultdict(float,
                         {'toas': 3.0, '__total': 6.0, 'väljas': 3.0}),
             'külm': defaultdict(float, {'toas': 4.0, '__total': 4.0})})

Nüüd vaatame, millise tõenäosusega tehakse üks või teine otsus mingite etteantud tingimuste (pilvisus, temperatuur) puhul. Paneme tähele, et see ei sõltu mitte ainult otsuse tegemise, vaid ka kõigi selliste tingimuste esinemise arvust (sellisel tingimusel tehtud selliste otsuste arv jagatud kõigi selliste tingimuste esinemise arvuga).


In [ ]:
test = ["selge", "soe"]

# Pilvisuse seisukohalt...
otsus = ""
parim = -1
for k, v in pDecisionGivenCondition[test[0]].items():
    #print(k, v)
    # Jagame variandi esinemiskordade arvu kõigi kordade arvuga
    p = v / pDecisionGivenCondition[test[0]]["__total"]
    if p > parim and k != "__total":
        parim = p
        otsus = k

print(otsus, parim)

# Temperatuuri seisukohalt...
otsus = ""
parim = -1
for k, v in pDecisionGivenCondition1[test[1]].items():
    #print(k, v)
    p = v / pDecisionGivenCondition1[test[1]]["__total"]
    if p > parim and k != "__total":
        parim = p
        otsus = k

print(otsus, parim)

toas 0.8
toas 0.5


* Kumb mudel tundub meie praeguste andmete puhul olevat parem?  
* Mis juhtuks, kui me testiksime andmetega ["vihmane", "külm"]?

## Scikit-learn ja Pandas

Selle asemel, et käsitsi tõenäosusjaotusi arvutada, vajame suuremate andmetega tegutsemiseks juba valmis vahendeid statistiliseks masinõppeks. Sel korral kasutame nii praktikumis kui kodutöö puhul moodulit scikit-learn (http://scikit-learn.org/). Scikit-learn (mida saab importida käsuga "import sklearn") võiks koos NumPy ja SciPy moodulitega nii Colabis kui ka Anacondas juba olemas olla (vastasel juhul saab kasutada *'conda install scikit-learn'*), kui ei, saab selle [installida](http://scikit-learn.org/stable/install.html). Õpetusi scikit-learni jaoks leiate aadressilt http://scikit-learn.org/stable/tutorial.  

Kuigi Scikit-learnis on endas ka näidisandmestikke, siis sageli on meie andmed näiteks csv-failide kujul. Tegeleme järgnevalt failis esitatud andmetega: laeme Moodle'ist alla faili cars.csv ning loeme sealt andmed sisse.

Andmete käsitlemiseks kasutame teeki [Pandas](https://pandas.pydata.org/docs/user_guide/index.html), peamiselt selle DataFrame'e.

In [ ]:
# Impordime moodulid
import sklearn
import pandas as pd
# Loeme sisse csv.faili
failinimi = 'cars.csv'
autod = pd.read_csv(failinimi, delimiter=";")
# Trükime ekraanile andmete mõõtmed
print(autod.shape)
# Loeme andmete tunnused
tunnused = [i for i in autod]


(398, 7)


In [ ]:
# Vaatame andmetele sisse
autod.head()

,mpg,cylinders,horsepower,weight,acceleration,name,brand
0,18.0,8.0,130.0,3504,12.0,chevrolet chevelle malibu,chevrolet
1,15.0,8.0,165.0,3693,11.5,buick skylark 320,buick
2,18.0,8.0,150.0,3436,11.0,plymouth satellite,plymouth
3,16.0,8.0,150.0,3433,12.0,amc rebel sst,amc
4,17.0,8.0,140.0,3449,10.5,ford torino,ford


Tegemist on andmetega vanade autode kohta:
- mpg - kütusekulu (*miles per gallon*)
- cylinders - silindrite arv
- horsepower - võimsus (hobujõududes)
- weight - kaal 
- acceleration - kiirendus
- name - mudeli nimi
- brand - automark

In [ ]:
autod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     395 non-null    float64
 2   horsepower    392 non-null    float64
 3   weight        398 non-null    int64  
 4   acceleration  398 non-null    float64
 5   name          398 non-null    object 
 6   brand         398 non-null    object 
dtypes: float64(4), int64(1), object(2)
memory usage: 21.9+ KB


In [ ]:
autod.describe()

,mpg,cylinders,horsepower,weight,acceleration
count,398.000000,395.000000,392.000000,398.000000,398.000000
mean,23.514573,5.465823,104.469388,2970.424623,15.568090
std,7.815984,1.702707,38.491160,846.841774,2.757689
min,9.000000,3.000000,46.000000,1613.000000,8.000000
25%,17.500000,4.000000,75.000000,2223.750000,13.825000
50%,23.000000,4.000000,93.500000,2803.500000,15.500000
75%,29.000000,8.000000,126.000000,3608.000000,17.175000
max,46.600000,8.000000,230.000000,5140.000000,24.800000


* Mis tüüpi tunnused ja väärtused on?

* Kas andmestikus on puuduvaid väärtusi?

In [ ]:
# Saab vaadata ka vaid ühte tulpa korraga
autod.brand

0      chevrolet
1          buick
2       plymouth
3            amc
4           ford
         ...    
393         ford
394           vw
395        dodge
396         ford
397        chevy
Name: brand, Length: 398, dtype: object

In [ ]:
# või
autod["brand"]

0      chevrolet
1          buick
2       plymouth
3            amc
4           ford
         ...    
393         ford
394           vw
395        dodge
396         ford
397        chevy
Name: brand, Length: 398, dtype: object

In [ ]:
autod[["brand","name"]]

,brand,name
0,chevrolet,chevrolet chevelle malibu
1,buick,buick skylark 320
2,plymouth,plymouth satellite
3,amc,amc rebel sst
4,ford,ford torino
...,...,...
393,ford,ford mustang gl
394,vw,vw pickup
395,dodge,dodge rampage
396,ford,ford ranger


In [ ]:
# Täpsemaid ridu saab vaadata nt. loc abil
autod.loc[5:10]

,mpg,cylinders,horsepower,weight,acceleration,name,brand
5,15.0,8.0,198.0,4341,10.0,ford galaxie 500,ford
6,14.0,8.0,220.0,4354,9.0,chevrolet impala,chevrolet
7,14.0,8.0,215.0,4312,8.5,plymouth fury iii,plymouth
8,14.0,8.0,225.0,4425,10.0,pontiac catalina,pontiac
9,15.0,8.0,190.0,3850,8.5,amc ambassador dpl,amc
10,15.0,8.0,170.0,3563,10.0,dodge challenger se,dodge


In [ ]:
autod.loc[30]

mpg                            28.0
cylinders                       4.0
horsepower                     90.0
weight                         2264
acceleration                   15.5
name            chevrolet vega 2300
brand                     chevrolet
Name: 30, dtype: object

In [ ]:
## Nii saab uusi tulpi lisada
# Väärtustamisel saab ette anda ühe väärtuse (kõigile ridadele sama) või õige pikkusega listi
autod["temp"] = "tere"
autod.head()


,mpg,cylinders,horsepower,weight,acceleration,name,brand,temp
0,18.0,8.0,130.0,3504,12.0,chevrolet chevelle malibu,chevrolet,tere
1,15.0,8.0,165.0,3693,11.5,buick skylark 320,buick,tere
2,18.0,8.0,150.0,3436,11.0,plymouth satellite,plymouth,tere
3,16.0,8.0,150.0,3433,12.0,amc rebel sst,amc,tere
4,17.0,8.0,140.0,3449,10.5,ford torino,ford,tere


In [ ]:
## Tegelikult me seda tulpa ei taha
autod = autod.drop("temp",axis=1)

## Tõenäosuslik mudel ennustamiseks

Kuna andmed on meil olemas, siis püüame nende abil midagi ennustada. Esimese asjana võtame ette auto margi (*brand*). Kuna see on nominaalne tunnus, siis ennustame seda Scikit-learnis olemasolevate klassifitseerimismudelite abil. Et asi lihtsam oleks, võtame esialgu ette ainult 4 automarki.

In [ ]:
# Mis margid üldse mis koguses esindatud on?
print(autod.groupby('brand').size())

33


* Mitu erinevat klassi (automarki) on andmestikus?

* Kas klassid on tasakaalus?

In [ ]:
print(autod.groupby('brand').size().size)
print('pigem mitte')

33
pigem mitte


In [ ]:
# Võtame siit omal valikul 4 suuremat rühma
miniautod = autod.loc[autod['brand'].isin(['ford','chevrolet','toyota','plymouth'])]
# Mitu kirjet meile jäi?
print(miniautod.shape)
# NB! reaindeksid on algse DataFrame'i juurest
miniautod

(152, 7)


,mpg,cylinders,horsepower,weight,acceleration,name,brand
0,18.0,8.0,130.0,3504,12.0,chevrolet chevelle malibu,chevrolet
2,18.0,8.0,150.0,3436,11.0,plymouth satellite,plymouth
4,17.0,8.0,140.0,3449,10.5,ford torino,ford
5,15.0,8.0,198.0,4341,10.0,ford galaxie 500,ford
6,14.0,8.0,220.0,4354,9.0,chevrolet impala,chevrolet
...,...,...,...,...,...,...,...
389,22.0,6.0,112.0,2835,14.7,ford granada l,ford
390,32.0,4.0,96.0,2665,13.9,toyota celica gt,toyota
392,27.0,4.0,90.0,2950,17.3,chevrolet camaro,chevrolet
393,27.0,4.0,86.0,2790,15.6,ford mustang gl,ford


### Treening- ja testandmed
Jagame andmed treening- ja testkorpuseks ning eraldame mõlema puhul tingimustena kasutatavate ja ennustatavate tunnuste väärtused. 
NB! Tulevikus tuleb antud andmed jagada sageli ise treening-, arendus- ja testandmeteks (*train*, *development*, *test*): 
- treeningandmete (train) põhjal õpetame;
- arendusandmete (dev) põhjal kontrollime tulemust ja parandame oma algoritmi, kuid nii muutuvad arendusandmed ühtlasi kaudselt ka treeningandmeteks;
- testandmete (test) testime tulemust lõplikult.

Rusikareegel on, et nende andmete suuruste vahekord on 60%/20%/20%. Jagamine pole lihtne ülesanne: valel jagamisel võime saada sellise tulemuse, kus arendusandmed annavad treeningandmetel suurepäraseid tulemusi, kuid testandmetel mitte vms. Siinkohal tuleks arvestada kindlasti klasside tasakaaluga nii treening- kui ka testkorpuses. Selleks tuleks kasuks muuhulgas nt andmed **läbi segada** - kes teab, andmestik võib olla tunnuste väärtuste järgi sorteeritud.


In [ ]:
# Segame läbi
miniautod = sklearn.utils.shuffle(miniautod, random_state=40) 

# Mille põhjal ennustada
tunnused = ['mpg','weight','acceleration']
# Mida ennustada
ennustatav = 'brand'

# Võtame esimesed 30 kirjet testimiseks, ülejäänud treenimiseks.
# Üldine notatsioon on y = f(X) ehk ennustatav väärtus on y ja õppimistunnused on X.
X_train = miniautod[30:][tunnused]
y_train = miniautod[30:][ennustatav]

X_test = miniautod[:30][tunnused]
y_test = miniautod[:30][ennustatav]

# Mis kujul andmed meil on?
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(122, 3)
(122,)
(30, 3)
(30,)


### Õppimine ja testimine
Impordime mõned masinõppe meetodid:
- [Logistiline regressioon](http://scikit-learn.org/stable/modules/linear_model.html#logistic-regression)
- [KNeighborsClassifier](http://scikit-learn.org/stable/modules/neighbors.html)
- [(Gaussian) Naiivne Bayes](http://scikit-learn.org/stable/modules/naive_bayes.html)
- [Otsustuspuu](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier)

Kasutame neid treenimisel (*fit*), klassifitseerime saadud mudeli põhjal testhulgas olevaid kirjeid ja arvutame täpsuse (*score*). Kuna treening- ja testhulgad on erinevad (juhusliku järjestamise tõttu enne vastavat jagamist), siis on ka täpsused erinevad. Paneme tähele, et alltoodud koodis on meetodid vaikeväärtustega (ehk parameetreid pole määratud). Uurige vaikeparameetreid (mis trükitakse välja käsuga näiteks "print(neighbors.KNeighborsClassifier())") ja proovige neid muuta, näiteks:
- neighbors.KNeighborsClassifier(n_neighbors=4).fit(train_X, train_y).score(test_X, test_y))

Loetelu erinevatest masinõppemeetoditest leiate aadressilt http://scikit-learn.org/stable/supervised_learning.html Nendest saate valida midagi oma koduse töö jaoks, uurige ka parameetreid. 




In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

#### Õppimine

In [ ]:
# Algväärtusta mudel
mudel = LogisticRegression()

# Treeni treeningandmetel
mudel.fit(X_train, y_train)

LogisticRegression()

#### Ennustamine

In [ ]:
# Anna mudelile ette uued õppimistunnused ja ennusta
# NB! X_train ja X_test peavad sisaldama samal kujul tunnuseid
y_pred = mudel.predict(X_test)
y_pred

array(['ford', 'toyota', 'ford', 'ford', 'toyota', 'toyota', 'toyota',
       'ford', 'ford', 'toyota', 'ford', 'ford', 'ford', 'ford', 'ford',
       'ford', 'ford', 'ford', 'toyota', 'ford', 'toyota', 'ford',
       'toyota', 'ford', 'ford', 'ford', 'ford', 'ford', 'ford', 'toyota'],
      dtype=object)

In [ ]:
# Võime võrrelda saadud tulemusi õigete vastustega
print("õige\tennustatud")
print()
for paar in list(zip(y_test,y_pred)):
    print(paar[0],"\t",paar[1])

õige	ennustatud

chevrolet 	 ford
toyota 	 toyota
ford 	 ford
ford 	 ford
chevrolet 	 toyota
toyota 	 toyota
chevrolet 	 toyota
chevrolet 	 ford
chevrolet 	 ford
toyota 	 toyota
plymouth 	 ford
ford 	 ford
plymouth 	 ford
ford 	 ford
ford 	 ford
ford 	 ford
chevrolet 	 ford
chevrolet 	 ford
ford 	 toyota
ford 	 ford
toyota 	 toyota
ford 	 ford
plymouth 	 toyota
ford 	 ford
chevrolet 	 ford
ford 	 ford
plymouth 	 ford
chevrolet 	 ford
ford 	 ford
toyota 	 toyota


#### Meetrikud
Mudeli headuse hindamiseks on tavaliselt olnud täpsus (accuracy). Tegelikult on erinevaid headuse näitajaid väga palju. Näiteks täpsust on hea kasutada, kui on klassifitseerimine, aga pole väga hea, kui tegeleme regressiooniga. Näiteks maja hinna ennustamisel, kus õige väärtus on 100, on valed nii ennustused 92 kui ka 99 on valed, aga 99 on 100-le päris lähedal ja seega parem hinnang kui 92. 

**Täpsus**

In [ ]:
import numpy as np
from sklearn.metrics import  accuracy_score

print("Täpsus (käsitsi) on: %.2f." % np.mean(y_pred == y_test))
# Või siis scikit-learni meetodit kasutades
print("Täpsus (sci-kit) on: %.2f." % accuracy_score(y_test, y_pred))

Täpsus (käsitsi) on: 0.53.
Täpsus (sci-kit) on: 0.53.


**F-skoor: täpsus (*precision*) ja saagis (*recall*)**

In [ ]:
from sklearn.metrics import  classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

   chevrolet       0.00      0.00      0.00         9
        ford       0.52      0.92      0.67        12
    plymouth       0.00      0.00      0.00         4
      toyota       0.56      1.00      0.71         5

    accuracy                           0.53        30
   macro avg       0.27      0.48      0.35        30
weighted avg       0.30      0.53      0.39        30



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Segadusmaatriks**

Kas midagi on millekski valesti klassifitseeritud? Kui jah, siis mis?

In [ ]:
from sklearn.metrics import confusion_matrix
# Meeldetuletuseks: need olid ennustatavad klassid - järjekorra saame treeninud mudeli käest
print(mudel.classes_)
# Segadusmaatriks ise
# Read: tegelikud väärtused
# Tulbad: ennustatud väärtused
print(confusion_matrix(y_test, y_pred))

['chevrolet' 'ford' 'plymouth' 'toyota']
[[ 0  7  0  2]
 [ 0 11  0  1]
 [ 0  3  0  1]
 [ 0  0  0  5]]


## Ülesanne 1
Koostage mudelid ka ülejäänud kolme ülaltoodud meetodi põhjal. Kas tulemused tulid paremad või halvemad?

In [ ]:
mudel_2 = KNeighborsClassifier()
mudel_2.fit(X_train, y_train)

mudel_3 = GaussianNB()
mudel_3.fit(X_train, y_train)

mudel_4 = DecisionTreeClassifier()
mudel_4.fit(X_train, y_train)

DecisionTreeClassifier()

In [ ]:
y_pred_2 = mudel_2.predict(X_test)
y_pred_3 = mudel_3.predict(X_test)
y_pred_4 = mudel_4.predict(X_test)

In [ ]:
print("Lähimate naabrite täpsus on: %.2f." % accuracy_score(y_test, y_pred_2))
print("Naiivse Bayesi täpsus on: %.2f." % accuracy_score(y_test, y_pred_3))
print("Otsustuspuu täpsus on: %.2f." % accuracy_score(y_test, y_pred_4))

Lähimate naabrite täpsus on: 0.40.
Naiivse Bayesi täpsus on: 0.53.
Otsustuspuu täpsus on: 0.40.


In [ ]:
print(mudel_4.classes_)
print(confusion_matrix(y_test, y_pred_4))

['chevrolet' 'ford' 'plymouth' 'toyota']
[[3 3 1 2]
 [3 5 2 2]
 [1 2 1 0]
 [0 1 1 3]]


### Masinõpe - regressioon
Mudelite kirjeldustele otsa vaadates on näha, et osad neist on mõeldud klassifitseerimiseks, osad regressiooniks. Näiteks lähimate naabrite puhul on meil klassifitseerimiseks [*KNeighborsClassifier*](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier), mille vaste regressiooniülesande puhul oleks [*KNeighborsRegressor*](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html#sklearn.neighbors.KNeighborsRegressor). Regressiooni puhul mäletatavasti ennustasime mingit arvu pidevas vahemikus. Püüame ennustada kiirendust (*acceleration*) auto kaalu ja kütusekulu põhjal.

In [ ]:
autod = sklearn.utils.shuffle(autod, random_state=40) 

# Mille põhjal ennustada
tunnused = ['mpg','weight']
# Mida ennustada
ennustatav = 'acceleration'

# Võtame esimesed 50 kirjet testimiseks, ülejäänud treenimiseks.
# Üldine notatsioon on y = f(X) ehk ennustatav väärtus on y ja õppimistunnused on X.
X_train = autod[50:][tunnused]
y_train = autod[50:][ennustatav]

X_test = autod[:50][tunnused]
y_test = autod[:50][ennustatav]


In [ ]:
from sklearn.linear_model import LinearRegression
mudel = LinearRegression()
mudel.fit(X_train, y_train)
y_pred = mudel.predict(X_test)

In [ ]:
print("Mille põhjal ennustasime:",X_test[:1].values)
print("Tegelik väärtus:",y_test[:1].values)
print("Ennustatud väärtus:",y_pred[0])

Mille põhjal ennustasime: [[  26. 1963.]]
Tegelik väärtus: [15.5]
Ennustatud väärtus: 16.40926300525507


#### Meetrikud

Loomulikult ei saa säärasel puhul rääkida enam samasugusest täpsusest kui oli klassifitseerimisel. Küll aga saame vaadata seda, kui palju mudel ennustamisel numbriliselt eksis.

Erinevate meetrikute täpsemaid selgitusi võib leida [siit](https://scikit-learn.org/stable/modules/model_evaluation.html)

In [ ]:
from sklearn.metrics import max_error, mean_squared_error

print("Suurim erinevus:",max_error(y_test,y_pred))
# Keskmine erinevus - võetud ruutu, et erineda võiks nii üles kui alla
print("MSE: %.4f." % mean_squared_error(y_test, y_pred))
# Eelmise ruutjuur
print("RMSE: %.4f." % np.sqrt(mean_squared_error(y_test, y_pred)))

Suurim erinevus: 9.161649986821166
MSE: 7.8192.
RMSE: 2.7963.



Püüame nüüd ennustada auto kiirendust (*acceleration*) mootori silindrite arvu (*cylinders*), võimsuse (*horsepower*) ja kaalu (*weight*) abil.

In [ ]:
autod = sklearn.utils.shuffle(autod, random_state=40) 

# Mille põhjal ennustada
tunnused = ['cylinders','horsepower','weight']
# Mida ennustada
ennustatav = 'acceleration'

# Võtame esimesed 50 kirjet testimiseks, ülejäänud treenimiseks.
# Üldine notatsioon on y = f(X) ehk ennustatav väärtus on y ja õppimistunnused on X.
X_train = autod[50:][tunnused]
y_train = autod[50:][ennustatav]

X_test = autod[:50][tunnused]
y_test = autod[:50][ennustatav]

mudel = LinearRegression()
mudel.fit(X_train, y_train)
y_pred = mudel.predict(X_test)
print("MSE: %.4f." % mean_squared_error(y_test, y_pred))

MSE: 3.7118.


## Eeltöötlus

Vt. ka http://scikit-learn.org/stable/modules/preprocessing.html 

In [ ]:
autod.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 398 entries, 266 to 375
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     395 non-null    float64
 2   horsepower    398 non-null    float64
 3   weight        398 non-null    int64  
 4   acceleration  398 non-null    float64
 5   name          398 non-null    object 
 6   brand         398 non-null    object 
dtypes: float64(4), int64(1), object(2)
memory usage: 41.0+ KB


### Puuduvad andmed
Sageli on andmed puudulikud: mõni väärtus lihtsalt puudub või mõnd pole teada (mõne filmi režissöör (director) või eelarve, mõne autori mootori võimsus vms). Üheks variandiks on ära kustutada kõik kirjed, kus mõni väärtus puudub, kuid siis võime kaotada ära liiga suure hulga andmeid. Paremini sobivaks variandiks on puuduvate andmete mingil viisil lisamine. Arvuliste väärtuste puhul võime puuduvaks väärtuseks lisada näiteks sama tunnuse ülejäänud väärtuste keskmine või mediaan. Nominaalsete tunnuste puhul võime lisada uue väärtuse, mida senini pole selle tunnuse puhul kasutatud (*other*) või kasutame kõige sagedasemat väärtust. 

Nagu autode andmestiku kirjeldusest näeme, on puudu kuue auto mootori võimsust (*horsepower*) (398-392=6) ja kolme auto silindrite arvu (*cylinders*) (398-395=3) tähistavad andmed, need on tähistatud praegu lühendiga "NA" (*Not Available*). Kui me prooviksime praegu masinõppe erinevaid meetodeid kasutada, antakse meile viga. Seega peaksime puuduvad väärtused siiski lisama, kuid vajalik on see nende tunnuste puhul, mida me hiljem analüüsis kasutame. Võimsuse puhul on meil tegemist numbrilise väärtusega ja seepärast püüame kasutada näiteks teiste andmete keskmist väärtust. Keskmist võiksime arvutada pikema koodi abil, kuid meil on olemas ka moodul [*numpy*](https://docs.scipy.org/doc/numpy-1.13.0/reference/index.html) ja selles meetod [*mean*](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.mean.html#numpy.mean). Kuna me teame, et puuduv väärtus on meil kirjeldatud muidu numbrilises tulbas stringina "NA", siis sobib siin kasutada [meetodit isnan](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.isnan.html) (NaN = not a number).

In [ ]:
import numpy as np

keskmine = round(np.mean(autod["horsepower"]))
print("Keskmine:",keskmine)
for i in range(len(autod["horsepower"])):
    if np.isnan(autod["horsepower"][i]):
        autod.at[i,'horsepower'] = keskmine
#print(autod["horsepower"])

Keskmine: 104


Kopeerime siia kogu senise koodi ilma kommentaarideta, et edasine kood oleks lühem. Lisame puuduvad andmed, kuid seekord automaatselt scikit-learni enda vahenditega (vt. impute, https://scikit-learn.org/stable/modules/impute.html). Seejärel kontollime, kas meil on jätkuvalt puuduvaid väärtusi.


In [ ]:
import sklearn
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

autod = pd.read_csv(failinimi, delimiter=";")
autod.info()

# Lisame sklearni meetodi abil võimsust märkivateks puuduvateks väärtusteks olemasolevate keskmise väärtuse
# Keskmise (mitte kõige sagedasema väärtuse) seepärast, et tegemist on pidevate andmetega
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
autod["horsepower"] = imp.fit_transform(autod[["horsepower"]])

# Lisame sklearni meetodi abil silindrite arvu märkivateks puuduvateks väärtusteks kõige sagedasema väärtuse
# Kõige sagedasem (mitte keskmine) seepärast, et tegemist pole pidevate andmetega (4,7 silindriga mootorit pole olemas)
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
autod["cylinders"] = imp.fit_transform(autod[["cylinders"]])

#print(autod)''
print("\nJa pärast puuduvate väärtuste asendamist:")
autod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     395 non-null    float64
 2   horsepower    392 non-null    float64
 3   weight        398 non-null    int64  
 4   acceleration  398 non-null    float64
 5   name          398 non-null    object 
 6   brand         398 non-null    object 
dtypes: float64(4), int64(1), object(2)
memory usage: 21.9+ KB

Ja pärast puuduvate väärtuste asendamist:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    float64
 2   horsepower    398 non-null    float64
 3   weight        398 non-null    int64  
 4   acceleration  398 non-null    float6

Testandmete puhul tuleb samuti puuduvad andmed asendada. Me ei saa seda teha testandmete keskmise põhjal, sest sel juhul poleks puuduv väärtus testandmetel enam samal viisil kodeeritud, kui see oli treeningandmetel.


In [ ]:
# Loeme uuesti sisse, kuna enne said andmed igal pool asendatud
autod2 = pd.read_csv(failinimi, delimiter=";")
train = autod2[50:]
test = autod2[:50]

# Puuduvate hobujõuandmetega kirjed
autod2.loc[autod2['horsepower'].isna()]

,mpg,cylinders,horsepower,weight,acceleration,name,brand
32,25.0,4.0,NaN,2046,19.0,ford pinto,ford
126,21.0,6.0,NaN,2875,17.0,ford maverick,ford
330,40.9,4.0,NaN,1835,17.3,renault lecar deluxe,renault
336,23.6,4.0,NaN,2905,14.3,ford mustang cobra,ford
354,34.5,4.0,NaN,2320,15.8,renault 18i,renault
374,23.0,4.0,NaN,3035,20.5,amc concord dl,amc


In [ ]:
#Salvestame testhulgas olevate puuduvate väärtustega kirjete indeksid
puuduvad = test.loc[test['horsepower'].isna()].index.values
puuduvad = list(puuduvad)

In [ ]:
# Julmalt testandmete põhjal saadud tulemus 
impHP = SimpleImputer(missing_values=np.nan, strategy='mean')
train["horsepower"] = impHP.fit_transform(train[["horsepower"]])
test['horsepower'] = impHP.fit_transform(test[["horsepower"]])
print("Kui võtta aluseks testandmetel saadud keskmine")
print(test.loc[puuduvad])
print()

# Vaikimisi muudetakse ka autod2 DataFrame'i. 
# Algsete andmete samiseks oleks tulnud temast koopia teha.
# Aga võime praegu ka lihtsalt uuesti sisse lugeda.
autod2 = pd.read_csv(failinimi, delimiter=";")
test = autod2[:50]
print("Nullime oma testandmed vahepeal ära")
print(test.loc[puuduvad])
print()

# Loome uue asendaja
impHP = SimpleImputer(missing_values=np.nan, strategy='mean')
# Üksnes imputeri treenimiseks, ilma asendusteta
impHP.fit(train[["horsepower"]])
test = autod2[:50]
test['horsepower'] = impHP.transform(test[["horsepower"]])
print("Treeningandmetel õpetatud imputeri asendustega")
print(test.loc[puuduvad])

Kui võtta aluseks testandmetel saadud keskmine
     mpg  cylinders  horsepower  weight  acceleration        name brand
32  25.0        4.0  136.265306    2046          19.0  ford pinto  ford

Nullime oma testandmed vahepeal ära
     mpg  cylinders  horsepower  weight  acceleration        name brand
32  25.0        4.0         NaN    2046          19.0  ford pinto  ford

Treeningandmetel õpetatud imputeri asendustega
     mpg  cylinders  horsepower  weight  acceleration        name brand
32  25.0        4.0   99.927114    2046          19.0  ford pinto  ford


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

* Mis juhtuks siis, kui jätta esimesel juhul treeningandmetel õppimata?
* Mis väärtusega asendatakse treeninghulga puuduvad väärtused?

Mõningal juhul saab puuduvate väärtuste asendamisega tegelda otse DataFrame'is Pandase vahenditega - aga ka sel juhul tuleb olla ettevaatlik treening- ja testhulkade väärtustega.

In [ ]:
autod2 = pd.read_csv(failinimi, delimiter=";")
autod2["horsepower"].fillna(autod2.horsepower.mean(),inplace=True)
print(autod2.info())
print()
print(autod2.loc[puuduvad]) #32

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     395 non-null    float64
 2   horsepower    398 non-null    float64
 3   weight        398 non-null    int64  
 4   acceleration  398 non-null    float64
 5   name          398 non-null    object 
 6   brand         398 non-null    object 
dtypes: float64(4), int64(1), object(2)
memory usage: 21.9+ KB
None

     mpg  cylinders  horsepower  weight  acceleration        name brand
32  25.0        4.0  104.469388    2046          19.0  ford pinto  ford


## Ülesanne 2
Ennusta enne valitud väärtuste põhjal auto kiirendust. Kas tulemused olid nende väärtuste abil paremad?

In [ ]:
autod = sklearn.utils.shuffle(autod, random_state=40) 

# Mille põhjal ennustada
tunnused = ['cylinders','horsepower','weight']
# Mida ennustada
ennustatav = 'acceleration'

# Võtame esimesed 50 kirjet testimiseks, ülejäänud treenimiseks.
# Üldine notatsioon on y = f(X) ehk ennustatav väärtus on y ja õppimistunnused on X.
X_train = autod[50:][tunnused]
y_train = autod[50:][ennustatav]

X_test = autod[:50][tunnused]
y_test = autod[:50][ennustatav]

mudel = LinearRegression()
mudel.fit(X_train, y_train)
y_pred = mudel.predict(X_test)
print("MSE: %.4f." % mean_squared_error(y_test, y_pred))

MSE: 3.4264.


## Numbriliste väärtuste teisendamine nominaalseteks (väärtuste klassifitseerimine)
Mõnikord on probleemiks on andmete väike hulk, kuid ka see, et mõned tingimusteks olevad väärtused on numbrilised ja pidevad. Proovime numbrilisi väärtusi kategoriseerida. Selline teisendamine on reeglina kadudega. 

In [ ]:
# Kategoriseerime võimsuse normaliseeritud väärtusi
numOfCat = 3
horsepower_cat = pd.cut(autod['horsepower'], numOfCat, labels=range(numOfCat))
autod['horsepower_cat'] = horsepower_cat

In [ ]:
print(autod[['horsepower','horsepower_cat']])

     horsepower horsepower_cat
266        68.0              0
359        80.0              0
373        92.0              0
234        88.0              0
317        78.0              0
..          ...            ...
393        86.0              0
303        65.0              0
56         70.0              0
368        88.0              0
375        74.0              0

[398 rows x 2 columns]


In [ ]:
autod.groupby("horsepower_cat").size()

horsepower_cat
0    263
1    103
2     32
dtype: int64

In [ ]:
# Kategoriseerime kaalu väärtusi
numOfCat = 3
weight, vahemikud = pd.cut(autod['weight'], numOfCat, labels=["kerge","keskmine","raske"],retbins=True)
autod['weight_cat'] = weight
print(autod[['horsepower','horsepower_cat','weight','weight_cat']])

     horsepower horsepower_cat  weight weight_cat
266        68.0              0    2155      kerge
359        80.0              0    3230   keskmine
373        92.0              0    2865   keskmine
234        88.0              0    2740      kerge
317        78.0              0    2188      kerge
..          ...            ...     ...        ...
393        86.0              0    2790   keskmine
303        65.0              0    2020      kerge
56         70.0              0    1955      kerge
368        88.0              0    2640      kerge
375        74.0              0    1980      kerge

[398 rows x 4 columns]


In [ ]:
autod.groupby("weight_cat").size()

weight_cat
kerge       195
keskmine    138
raske        65
dtype: int64

In [ ]:
# Bins näitab seda, mille pealt lahku löödi.
# Ära on toodud ka miinimum- ja maksimumväärtused.
print(vahemikud)

[1609.473      2788.66666667 3964.33333333 5140.        ]


## Nominaalsete väärtuste teisendamine numbrilisele kujule
Paljud meetodid ei luba töötada kategooriate ja nominaalsete tunnustega. Kuna nominaalsete väärtuste lihtne teisendamine numbriteks pole õige (teisendades näiteks soo puhul väärtuse "mees" 1-ga ja väärtuse "naine" 2-ga on need väärtused suuruse mõttes võrreldavad), siis kasutatakse nn "1-hot vektoreid": ühest tunnusest (näiteks "sugu") tehakse mitu (näiteks "mees" ja "naine") ning väärtustena on sel juhul 1-ga tähistatud see tunnus, mis on tõene:
- mehe puhul "mees" = 1, "naine" = 0
- naise puhul "mees" = 0, "naine" = 1

Autoandmete puhul võime mõelda näiteks silindrite arvu või brändi teisendamist numbrilisele kujule. Jah, silindrite arv on küll arv, kuid mitte pidev ning me ei arvesta 5,6 silindrilise mootoriga. 

Teeme teisendamist praegu auto brändi nimega. Kuna tulpade hulk on väga suur, on ekraanil tulemust keeruline jälgida, seepärast salvestame tulemused ka faili (http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_csv.html). 

In [ ]:
# Failikirjutamine
test.to_csv("test_results.csv", sep=';')

In [ ]:
onehot = pd.get_dummies(autod['brand'])
autod = autod.join(onehot)
#Kirjuta faili:
autod.to_csv("autod_results.csv", sep=';')
#print(autod)
autod.head()

,mpg,cylinders,horsepower,weight,acceleration,name,brand,horsepower_cat,weight_cat,amc,...,plymouth,pontiac,renault,saab,subaru,toyota,triumph,volkswagen,volvo,vw
266,30.0,4.0,68.0,2155,16.5,chevrolet chevette,chevrolet,0,kerge,0,...,0,0,0,0,0,0,0,0,0,0
359,28.1,4.0,80.0,3230,20.4,peugeot 505s turbo diesel,peugeot,0,keskmine,0,...,0,0,0,0,0,0,0,0,0,0
373,24.0,4.0,92.0,2865,16.4,ford fairmont futura,ford,0,keskmine,0,...,0,0,0,0,0,0,0,0,0,0
234,24.5,4.0,88.0,2740,16.0,pontiac sunbird coupe,pontiac,0,kerge,0,...,0,1,0,0,0,0,0,0,0,0
317,34.3,4.0,78.0,2188,15.8,audi 4000,audi,0,kerge,0,...,0,0,0,0,0,0,0,0,0,0


### Testandmete töötlus
Kuidas teisendatakse sel juhul testandmeid? Kui võtame testhulga ja treeninghulga, siis mis oleksid pärast sarnase töötluse kordamist nende kahe DataFrame'i mõõtmed?

In [ ]:
# Loeme andmed uuesti sisse, jaotame test- ja treeninghulgaks
autod2 = pd.read_csv(failinimi, delimiter=';')
train2 = autod2[50:]
test2 = autod2[:50]

In [ ]:
onehot_train = pd.get_dummies(train2['brand'])
onehot_test = pd.get_dummies(test2['brand'])
print("Tervel andmestikul saadud 1-hot vektorid",onehot.shape)
print("Treeningandmestikul saadud 1-hot vektorid",onehot_train.shape)
print("Testandmestikul saadud 1-hot vektorid",onehot_test.shape)


Tervel andmestikul saadud 1-hot vektorid (398, 33)
Treeningandmestikul saadud 1-hot vektorid (348, 32)
Testandmestikul saadud 1-hot vektorid (50, 17)


In [ ]:
test[:5]

,mpg,cylinders,horsepower,weight,acceleration,name,brand
0,18.0,8.0,130.0,3504,12.0,chevrolet chevelle malibu,chevrolet
1,15.0,8.0,165.0,3693,11.5,buick skylark 320,buick
2,18.0,8.0,150.0,3436,11.0,plymouth satellite,plymouth
3,16.0,8.0,150.0,3433,12.0,amc rebel sst,amc
4,17.0,8.0,140.0,3449,10.5,ford torino,ford


## Ülesanne 3

Teisendame onehot_testi selliseks, et tal oleks sama palju tulpi kui treeningandmetel (ja sama sisuga). Selleks tuleb  
1) visata välja tulbad, mida pole vaja  
2) lisada tulbad, mis on puudu - mis peaks siin olema puuduva väärtuse puhul?

In [ ]:
# mis tunnused on olemas treening- ja testhulgas
traintunnused = [i for i in onehot_train]
testtunnused = [i for i in onehot_test]

# Meie uus testhulk, kus on õiged onehot-tulbad juures
uus_onehot_test = pd.DataFrame()

print(traintunnused)
print(testtunnused)

#####
## TODO
#####
for tunnus in traintunnused:
    if tunnus in testtunnused:
        uus_onehot_test[tunnus] = onehot_test[tunnus]
    else:
        uus_onehot_test[tunnus] = 0

['amc', 'audi', 'bmw', 'buick', 'cadillac', 'capri', 'chevrolet', 'chevy', 'chrysler', 'datsun', 'dodge', 'fiat', 'ford', 'honda', 'mazda', 'mercedes', 'mercedes-benz', 'mercury', 'nissan', 'oldsmobile', 'opel', 'peugeot', 'plymouth', 'pontiac', 'renault', 'saab', 'subaru', 'toyota', 'triumph', 'volkswagen', 'volvo', 'vw']
['amc', 'audi', 'bmw', 'buick', 'chevrolet', 'chevy', 'datsun', 'dodge', 'ford', 'hi', 'mercury', 'peugeot', 'plymouth', 'pontiac', 'saab', 'toyota', 'volkswagen']


In [ ]:
uus_onehot_test.describe()

,amc,audi,bmw,buick,cadillac,capri,chevrolet,chevy,chrysler,datsun,...,plymouth,pontiac,renault,saab,subaru,toyota,triumph,volkswagen,volvo,vw
count,50.00000,50.000000,50.000000,50.000000,50.0,50.0,50.00000,50.000000,50.0,50.000000,...,50.000000,50.000000,50.0,50.000000,50.0,50.000000,50.0,50.000000,50.0,50.0
mean,0.14000,0.020000,0.020000,0.040000,0.0,0.0,0.14000,0.020000,0.0,0.040000,...,0.120000,0.080000,0.0,0.020000,0.0,0.040000,0.0,0.020000,0.0,0.0
std,0.35051,0.141421,0.141421,0.197949,0.0,0.0,0.35051,0.141421,0.0,0.197949,...,0.328261,0.274048,0.0,0.141421,0.0,0.197949,0.0,0.141421,0.0,0.0
min,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
25%,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
50%,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
75%,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
max,1.00000,1.000000,1.000000,1.000000,0.0,0.0,1.00000,1.000000,0.0,1.000000,...,1.000000,1.000000,0.0,1.000000,0.0,1.000000,0.0,1.000000,0.0,0.0


In [ ]:
# Vaatame ühte kirjet
uus_onehot_test.loc[20]

#NB! Ühel juhul kaotame ära tulpade järjestuse
#    (mis pole oluline seni, kuni oleme DataFrame'i kujul)

amc              0
audi             0
bmw              0
buick            0
cadillac         0
capri            0
chevrolet        0
chevy            0
chrysler         0
datsun           0
dodge            0
fiat             0
ford             0
honda            0
mazda            0
mercedes         0
mercedes-benz    0
mercury          0
nissan           0
oldsmobile       0
opel             0
peugeot          1
plymouth         0
pontiac          0
renault          0
saab             0
subaru           0
toyota           0
triumph          0
volkswagen       0
volvo            0
vw               0
Name: 20, dtype: int64

Ja muidugi saab kasutada näiteks Scikit-learni vahendit [OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html).

In [ ]:
train.head()

,mpg,cylinders,horsepower,weight,acceleration,name,brand
50,28.0,NaN,90.0,2123,14.0,opel 1900,opel
51,30.0,4.0,70.0,2074,19.5,peugeot 304,peugeot
52,30.0,4.0,76.0,2065,14.5,fiat 124b,fiat
53,31.0,4.0,65.0,1773,19.0,toyota corolla 1200,toyota
54,35.0,4.0,69.0,1613,18.0,datsun 1200,datsun


In [ ]:
from sklearn.preprocessing import OneHotEncoder
# 1-hot vektoriseerija, ignoreeri tundmatuid väärtusi
onehotEnc = OneHotEncoder(handle_unknown = 'ignore')
# Õpi
onehotEnc.fit(train[['brand']])
# Loo 1-hot vektorid
onehot = onehotEnc.transform(train[['brand']])
# Ennäe, need on õpitud treeningandmetel
print(onehot.shape)

# Kui on soovi, saab kättte ka tulpade nimed
print(onehotEnc.categories_)

(348, 32)
[array(['amc', 'audi', 'bmw', 'buick', 'cadillac', 'capri', 'chevrolet',
       'chevy', 'chrysler', 'datsun', 'dodge', 'fiat', 'ford', 'honda',
       'mazda', 'mercedes', 'mercedes-benz', 'mercury', 'nissan',
       'oldsmobile', 'opel', 'peugeot', 'plymouth', 'pontiac', 'renault',
       'saab', 'subaru', 'toyota', 'triumph', 'volkswagen', 'volvo', 'vw'],
      dtype=object)]


Vahel juhtub, et testhulga andmed tulevad meile ükshaaval. Tüüpiliselt näiteks siis, kui mudel juba kliendi käes on ja see tahab ühe konkreetse auto kohta midagi teada. Sel juhul tuleb andmetüüpidega ettevaatlik olla.

In [ ]:
print(type(autod))
print(type(autod.acceleration))
print(type(autod.loc[0]))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


OneHotEncoder rakendatuna **ühele testhulga kirjele** (mitte DataFrame'ile)

In [ ]:
# Teisendame ühe kirje DataFrame'iks - muidu oli ta Series
sample = pd.DataFrame(test.loc[20])
print(sample)
print()
# Transponeerime oma kirje, et algsed veerud oleksid ka temal veerud
sample= sample.T
print("Transponeeritult:\n",sample)

                       20
mpg                  25.0
cylinders             4.0
horsepower           87.0
weight               2672
acceleration         17.5
name          peugeot 504
brand             peugeot

Transponeeritult:
      mpg cylinders horsepower weight acceleration         name    brand
20  25.0       4.0       87.0   2672         17.5  peugeot 504  peugeot


In [ ]:
print("Teisendatav väärtus",sample['brand'])
print()
# Laseme sellele oma eelnevalt treenitud kodeerija peale
teisendus = onehotEnc.transform(sample[['brand']])
teisendus


Teisendatav väärtus 20    peugeot
Name: brand, dtype: object



<1x32 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [ ]:
print(teisendus)
print("Ja väärtus 1 on klassides",[(idx, onehotEnc.categories_[0][idx]) for idx in teisendus.indices])

  (0, 21)	1.0
Ja väärtus 1 on klassides [(21, 'peugeot')]


In [ ]:
# Kuidas uued väärtused DataFrame'iks saada
# Tulbanimed saab OneHotEncoderilt, reaineksid meie sample'ilt endalt.
sample_onehot = pd.DataFrame.sparse.from_spmatrix(teisendus, columns=onehotEnc.get_feature_names(), index=sample.index)
# Lisame uued tulbad vanadele juurde, pane reanumbri järgi kokku
sample_uus = pd.concat([sample, sample_onehot], axis=1)
sample_uus

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,mpg,cylinders,horsepower,weight,acceleration,name,brand,x0_amc,x0_audi,x0_bmw,...,x0_plymouth,x0_pontiac,x0_renault,x0_saab,x0_subaru,x0_toyota,x0_triumph,x0_volkswagen,x0_volvo,x0_vw
20,25.0,4.0,87.0,2672,17.5,peugeot 504,peugeot,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Natuke parema loetavuse huvides
sample_uus.iloc[0]
# iloc - lihtsalt loenda, ära reaindeksitest hooli
# või siis
# sample_uus.loc[20]

mpg                        25.0
cylinders                   4.0
horsepower                 87.0
weight                     2672
acceleration               17.5
name                peugeot 504
brand                   peugeot
x0_amc                        0
x0_audi                       0
x0_bmw                        0
x0_buick                      0
x0_cadillac                   0
x0_capri                      0
x0_chevrolet                  0
x0_chevy                      0
x0_chrysler                   0
x0_datsun                     0
x0_dodge                      0
x0_fiat                       0
x0_ford                       0
x0_honda                      0
x0_mazda                      0
x0_mercedes                   0
x0_mercedes-benz              0
x0_mercury                    0
x0_nissan                     0
x0_oldsmobile                 0
x0_opel                       0
x0_peugeot                  1.0
x0_plymouth                   0
x0_pontiac                    0
x0_renau

In [ ]:
print('coolcoolcool')

coolcoolcool
